In [ ]:
from tsfresh.feature_extraction import extract_features
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
from tsfresh.feature_extraction.settings import from_columns

import numpy as np
import pandas as pd

This notebooks illustrates the `"fc_parameters"` or `"kind_to_fc_parameters"` dictionaries.

For a detailed explanation, see also http://tsfresh.readthedocs.io/en/latest/text/feature_extraction_settings.html

## Construct a time series container

We construct the time series container that includes two sensor time series, _"temperature"_ and _"pressure"_, for two devices _"a"_ and _"b"_

In [ ]:
df = pd.DataFrame({"id": ["a", "a", "b", "b"], "temperature": [1,2,3,1], "pressure": [-1, 2, -1, 7]})
df

## The default_fc_parameters

Which features are calculated by tsfresh is controlled by a dictionary that contains a mapping from feature calculator names to their parameters. 
This dictionary is called `fc_parameters`. It maps feature calculator names (=keys) to parameters (=values). As keys, always the same names as in the tsfresh.feature_extraction.feature_calculators module are used.

In the following we load an exemplary dictionary

In [ ]:
settings_minimal = MinimalFCParameters() # only a few basic features
settings_minimal

This dictionary can passed to the extract method, resulting in a few basic time series beeing calculated:

In [ ]:
X_tsfresh = extract_features(df, column_id="id", default_fc_parameters = settings_minimal)
X_tsfresh.head()

By using the settings_minimal as value of the default_fc_parameters parameter, those settings are used for all type of time series. In this case, the `settings_minimal` dictionary is used for both _"temperature"_ and _"pressure"_ time series.

Now, lets say we want to remove the length feature and prevent it from beeing calculated. We just delete it from the dictionary.

In [ ]:
del settings_minimal["length"]
settings_minimal

Now, if we extract features for this reduced dictionary, the length feature will not be calculated

In [ ]:
X_tsfresh = extract_features(df, column_id="id", default_fc_parameters = settings_minimal)
X_tsfresh.head()

## The kind_to_fc_parameters

now, lets say we do not want to calculate the same features for both type of time series. Instead there should be different sets of features for each kind.

To do that, we can use the `kind_to_fc_parameters` parameter, which lets us finely specifiy which `fc_parameters` we want to use for which kind of time series:

In [ ]:
fc_parameters_pressure = {"length": None, 
                          "sum_values": None}

fc_parameters_temperature = {"maximum": None, 
                             "minimum": None}

kind_to_fc_parameters = {
    "temperature": fc_parameters_temperature,
    "pressure": fc_parameters_pressure
}

print(kind_to_fc_parameters)

So, in this case, for sensor _"pressure"_ both _"max"_ and _"min"_ are calculated. For the _"temperature"_ signal, the length and sum_values features are extracted instead.

In [ ]:
X_tsfresh = extract_features(df, column_id="id", kind_to_fc_parameters = kind_to_fc_parameters)
X_tsfresh.head()

So, lets say we lost the kind_to_fc_parameters dictionary. Or we apply a feature selection algorithm to drop 
irrelevant feature columns, so our extraction settings contain irrelevant features. 

In both cases, we can use the provided "from_columns" method to infer the creating dictionary from 
the dataframe containing the features

In [ ]:
recovered_settings = from_columns(X_tsfresh)
recovered_settings

Lets drop a column to show that the inferred settings dictionary really changes

In [ ]:
X_tsfresh.iloc[:, 1:]

In [ ]:
recovered_settings = from_columns(X_tsfresh.iloc[:, 1:])
recovered_settings

## More complex dictionaries

We provide custom fc_parameters dictionaries with greater sets of features.

The `EfficientFCParameters` contain features and parameters that should be calculated quite fastly:

In [ ]:
settings_efficient = EfficientFCParameters()
settings_efficient

The `ComprehensiveFCParameters` are the biggest set of features. It will take the longest to calculate

In [ ]:
settings_comprehensive = ComprehensiveFCParameters()
settings_comprehensive

You see those parameters as values in the fc_paramter dictionary? Those are the parameters of the feature extraction methods.

In detail, the value in a fc_parameters dicitonary can contain a list of dictionaries. Every dictionary in that list is one feature.

So, for example

In [ ]:
settings_comprehensive['large_standard_deviation']

would trigger the calculation of 20 different 'large_standard_deviation' features, one for r=0.05, for n=0.10 up to r=0.95.  Lets just take them and extract some features

In [ ]:
settings_value_count = {'large_standard_deviation': settings_comprehensive['large_standard_deviation']}
settings_value_count

In [ ]:
X_tsfresh = extract_features(df, column_id="id", default_fc_parameters=settings_value_count)
X_tsfresh.head()

The nice thing is, we actually contain the parameters in the feature name, so it is possible to reconstruct 
how the feature was calculated.

In [ ]:
from_columns(X_tsfresh)

This means that you should never change a column name. Otherwise the information how it was calculated can get lost.